In [2]:
import time

class Turbina():
    def __init__(self) -> None:
        self.Valvula = 0.0
        self.motorAux = False
        self.junta = False
        self.Q1 = False
        self.Q2 = False
        self.QE = False
        self.modoAuto = False
        self.freno = False
        self.RPM = 0.0
        self.friccion = 5.0
        
        self.aporteMotor = 0.0

        self.anteriores = []


    def update(self):
    # Si el motor auxiliar esta encendido y la junta de la turbina, el aporte de aceleración es 10 (o el numero que "self.aporteMotor" diga), 
    # de lo contrario se restablece a cero.
        if self.motorAux and self.junta:
            self.aporteMotor = 10
        else:
            self.aporteMotor = 0.0

    # Agregamos el aporte de los quemadores los cuales dependen de la apertura de la valvula, el parametro 
    # Valvula es el porcentaje de apertura de la valvula y es lo que deberemos variar para controlar la 
    # cantidad de aporte de aceleracion al motor que otorgan los quemadores. Se puede controlar tambien el 
    # aporte de aceleración en razon de cuantos quemadores se encuentren encendidos.
        if self.Q1 and self.Q2 and self.RPM > 478.0: # se agrega un minimo de velocidad ya que si la turbina está apagada por mas que los 
            # quemadores esten encendidos, no se puede generar una aceleración
            self.aporteQuemadores = self.Valvula * 1.0 #Este factor 1.0 es el valor a variar la apertura de la valvula

    # En esta ecuación controlamos el aporte de velocidad en el tiempo (RPM), se puede resolver mediante otras ecuaciones, como una
    # exponenecial, el resultado de alguna ecuación diferencial, etc., dependiendo de la magnitud de la fuerza
    # aplicada, por ejemplo, sumando la inercia de la masa, el radio de la turbina, el peso en Kg, etc.
    # Se encuentra implicito el tiempo de actualización (*tiempo) que multiplica el aporte de velocidad.
        self.RPM += self.aporteMotor + self.aporteQuemadores - self.friccion

        self.RPM -= self.friccion
        if self.RPM <= 0.0:
            self.RPM = 0.0

    def PID(self,input, Man_Auto = False, SetpointMan = 0.0, SetpointAuto = 0.0):
            """
                Calcula la salida de un controlador PID (Proporcional, Integral, Derivativo).

                Args:
                    Man_Auto (bool): Modo manual (True) o automático (False).
                    SetpointMan (float): Ignorado si Man_Auto es True.
                    SetpointAuto (float): El valor del setpoint en modo automático.

                Returns:
                    None

                El método calcula la salida del controlador PID utilizando el valor actual (input) y el setpoint (SetpointAuto).
                Se almacena el histórico de velocidades en self.anteriores y se limita a 100 elementos.
                Se calculan los componentes P, I y D del controlador y se suman para obtener la salida.
                La salida se limita al rango de 0 a 100.

                """
            if Man_Auto == False:     
                # Si el PID está en modo automático...

                # Almaceno el vector velocidad en una lista de 100 elementos.
                self.anteriores.append(input)
                if len(self.anteriores) > 100:
                    self.anteriores = self.anteriores[-100:]

                SP = SetpointAuto        
                E = SP - input
                self.error = E 
                
                #error es la diferencia entre lo que tengo, y mi setpoint actual. Usamos la lista para ello. 
                E_accu = [(SP - elem) for elem in self.anteriores[-20:]]
                self.error_accu = E_accu
                
                kP = 4.0
                kI = 0.00005
                kD = 0.01

                #La acción proporcional es el error multiplicado por una constante
                aP = self.error * kP
                
                #La acción integral es el área de los valores, dividido por la constante
                aI = (kP * (sum(self.error_accu) / (len(self.error_accu)*0.002) * kI))

                #La acción derivativa es la proyección a futuro (pendiente) del error, multiplicado por una constante
                if len(self.anteriores)>2:
                    aD = (self.error_accu[-1]-self.error_accu[-2])*kD*kP
                else:
                    aD = 0.0
                
                #sumamos las componentes de las acciones Proporcional, Integral y Derivativa
                Salida = self.Valvula + aP + aI + aD

                #Limitamos la válvula de salida
                if Salida < 0:
                    self.Valvula = 0
                elif Salida > 100:
                    self.Valvula = 100         
                else:
                    self.Valvula = Salida

            else:
                # Si estamos en modo "Manual", la válvula se pone en la posición que definimos en el setpoint.
                self.Valvula = SetpointMan

In [ ]:

TUR1 = Turbina()
TUR1.Q1 = True
TUR1.Q2 = True
TUR1.RPM = 1000.0
estados = range(0, 4)


try:
    while True:
        for i in estados:
            print(f"ESTADO: {i+1}")
        TUR1.update()
        print(f"VEL: {TUR1.RPM:.2f}, VALV: {TUR1.Valvula:.2f}")
        time.sleep(0.1)
    #Esta es la linea de codigo que setea el Lazo PID, aca usar un "if velocidad > 1750: el Man_Auto = False y SetpointAuto = 4600.0 RPMs"
        TUR1.PID(input=TUR1.RPM, Man_Auto=False, SetpointAuto=3000.0)
        TUR1.update()

        print(f"VEL 2: {TUR1.RPM:.2f}, VALV 2: {TUR1.Valvula:.2f}")
        time.sleep(0.1)
except KeyboardInterrupt:
    print("Simulación finalizada")